<a href="https://colab.research.google.com/github/Joyschool/2025_mirae_master/blob/main/2%EC%9D%BC%EC%B0%A8_%EB%B0%94%EC%9D%B4%EB%B8%8C%EC%BD%94%EB%94%A9_%EC%84%9C%EC%9A%B8%EC%8B%9C%EC%A7%80%ED%95%98%EC%B2%A0%EC%8A%B9%ED%95%98%EC%B0%A8%EC%9D%B8%EC%9B%90%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 바이브 코딩 - 서울시지하철승하차인원분석

- **(코랩에서)한글폰트 설치하기**

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 런타입 > 세션 다시 시작



---



## 1.데이터 준비



- 데이터 출처 : 서울 열린데이터광장
- URL :https://data.seoul.go.kr/dataList/OA-12914/S/1/datasetView.do
- 데이터 : 202505월 데이터,  CARD_SUBWAY_MONTH_202505.csv

## 2.분석 툴
- ChatGPT (무료 GPT-4o)
- Gemini 2.5 Flash (무료)

## 3. 바이브 코딩으로 데이터 분석하기

### 3-1.ChatGPT로 데이터 분석하기

- 시작->데이터 주고 무작정 데이터 분석 요청하기!

#### 1.날짜별 전체 승차/하차 인원수 추이 그래프

In [ ]:
import pandas as pd

# Load the uploaded CSV file
file_path = "CARD_SUBWAY_MONTH_202505.csv"
df = pd.read_csv(file_path, index_col=False, header=0) # 첫 컬럼 인덱스제외, 첫 행은 컬럼으로 사용

# Display the first few rows and basic info
df.head()
# df.info()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
# matplotlib에서 기본 폰트로 지정
plt.rcParams['font.family'] = 'NanumGothic'



# 날짜 형식 처리
# df['사용일자'] = pd.to_datetime(df['사용일자'], format='%Y%m%d')
# df['사용일자'] = df['사용일자'].dt.strftime('%Y-%m-%d')
# df['사용일자']

# 1. 날짜별 전체 승차/하차 인원수 추이
daily_summary = df.groupby('사용일자')[['승차총승객수', '하차총승객수']].sum()
daily_summary


# 그래프 스타일 설정
plt.figure(figsize=(14, 6))
sns.lineplot(data=daily_summary)
# daily_summary.plot(kind='bar', figsize=(14, 6))
plt.title("2025년 5월 지하철 이용 추이 (일별 총 승차/하차 인원수)")
plt.xlabel("날짜")
plt.ylabel("인원수")
plt.xticks(rotation=45)
# plt.tight_layout()
plt.grid(True)
plt.show()


#### 2.노선별 총 승하차 인원 상위/하위

In [ ]:
# 노선별로 총 승차/하차 인원을 합산
line_summary = df.groupby('노선명')[['승차총승객수', '하차총승객수']].sum()

# 총 이용객 수 컬럼 추가
line_summary['총이용객수'] = line_summary['승차총승객수'] + line_summary['하차총승객수']

# 상위 5개 노선
top_lines = line_summary.sort_values('총이용객수', ascending=False).head(5)

# 하위 5개 노선
bottom_lines = line_summary.sort_values('총이용객수').head(5)

top_lines, bottom_lines


#### 3.역별 이용객 수 상위 10곳

In [ ]:
# '사용일자'가 숫자인 행만 필터링 후 날짜로 변환
df = df[df['사용일자'].astype(str).str.isnumeric()]
df['사용일자'] = pd.to_datetime(df['사용일자'], format='%Y%m%d')

# 총 이용객수 컬럼 생성 (승차 + 하차)
df['총이용객수'] = df['승차총승객수'] + df['하차총승객수']

# 역별 총 이용객수 집계 후 상위 10개 추출
top10_stations = df.groupby('역명')['총이용객수'].sum().sort_values(ascending=False).head(10)

top10_stations


#### 4.대표값(총합, 평균 등) 정리

In [ ]:
import pandas as pd

# 전처리된 DataFrame이 df라고 가정

# 총합
total_boarding = df['승차총승객수'].sum()
total_alighting = df['하차총승객수'].sum()
total_usage = total_boarding + total_alighting

# 일일 평균 (일자 수 기준)
daily_avg_boarding = df.groupby('사용일자')['승차총승객수'].sum().mean()
daily_avg_alighting = df.groupby('사용일자')['하차총승객수'].sum().mean()
daily_avg_total = daily_avg_boarding + daily_avg_alighting

# 역별 평균 (역 기준)
station_avg_boarding = df.groupby('역명')['승차총승객수'].sum().mean()
station_avg_alighting = df.groupby('역명')['하차총승객수'].sum().mean()
station_avg_total = station_avg_boarding + station_avg_alighting

# 결과 정리
summary = {
    "총 승차 인원": total_boarding,
    "총 하차 인원": total_alighting,
    "총 이용객 수": total_usage,
    "일일 평균 승차 인원": daily_avg_boarding,
    "일일 평균 하차 인원": daily_avg_alighting,
    "일일 평균 이용객 수": daily_avg_total,
    "역당 평균 승차 인원": station_avg_boarding,
    "역당 평균 하차 인원": station_avg_alighting,
    "역당 평균 이용객 수": station_avg_total
}

# 결과 출력
for k, v in summary.items():
    print(f"{k}: {v:,.0f}명")




---



### 3-2.Gemini로 데이터 분석하기

- [주의!] Gemini 무료 버전은 TXT 유형의 데이터를 분석할 수 있다.
- 엑셀에서 데이터를 텍스트로 변환하고 데이터 분석 요청하기

#### 1.가장 붐비는 노선(승차 및 하차 총합 기준) TOP5

In [ ]:
import pandas as pd

# 1. 파일 경로 지정 (업로드된 파일이 현재 실행 환경에 있다고 가정)
file_path = 'CARD_SUBWAY_MONTH_202505.txt'

try:
    # 2. 탭(tab)으로 구분된 CSV 파일 읽기
    # encoding='cp949' 또는 'euc-kr'을 사용하는 경우가 많습니다.
    # 파일을 읽다가 인코딩 에러가 나면 다른 인코딩을 시도해 보세요.
    df = pd.read_csv(file_path, sep='\t', encoding='cp949')

    # 3. 필요한 컬럼의 데이터 타입 확인 및 변환
    # '승차총승객수'와 '하차총승객수'가 숫자로 되어 있는지 확인하고, 아니면 숫자로 변환합니다.
    # errors='coerce' 옵션을 사용하여 숫자로 변환할 수 없는 값은 NaN으로 만듭니다.
    df['승차총승객수'] = pd.to_numeric(df['승차총승객수'], errors='coerce')
    df['하차총승객수'] = pd.to_numeric(df['하차총승객수'], errors='coerce')

    # NaN 값 (변환 실패한 값)이 있다면 0으로 채웁니다.
    df = df.fillna(0)

    # 4. 노선별 총 승객수 계산
    # '승차총승객수'와 '하차총승객수'를 더하여 '총승객수' 컬럼을 만듭니다.
    df['총승객수'] = df['승차총승객수'] + df['하차총승객수']

    # '노선명'별로 '총승객수'를 합산합니다.
    total_passengers_by_line = df.groupby('노선명')['총승객수'].sum()

    # 5. 가장 붐비는 노선 TOP 5 선정
    top_5_lines = total_passengers_by_line.nlargest(5)

    # 6. 결과 출력
    print("가장 붐비는 노선 TOP 5 (승차 및 하차 총합 기준):")
    print(top_5_lines)

except FileNotFoundError:
    print(f"오류: '{file_path}' 파일을 찾을 수 없습니다. 파일이 올바른 위치에 있는지 확인해주세요.")
except Exception as e:
    print(f"데이터 처리 중 오류가 발생했습니다: {e}")

#### 2.TOP5 막대그래프로 표현하기

In [ ]:
#plt.figure(figsize=(10, 6)) # 그래프 크기 설정
top_5_lines.plot(kind='bar', color='skyblue') # 막대 그래프 그리기

# 그래프 제목 및 축 라벨 설정
plt.title('가장 붐비는 지하철 노선 TOP 5 (2025년 5월)', fontsize=16)
plt.xlabel('노선명', fontsize=12)
plt.ylabel('총 승객수', fontsize=12)

# Y축 숫자 표기법 변경 (천 단위 콤마)
plt.ticklabel_format(axis='y', style='plain') # 지수 표기법 대신 일반 숫자 표기
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
plt.show()

In [ ]:
try:
    # 2. 탭(tab)으로 구분된 CSV 파일 읽기
    # encoding='cp949' 또는 'euc-kr'을 사용하는 경우가 많습니다.
    # 파일을 읽다가 인코딩 에러가 나면 다른 인코딩을 시도해 보세요.
    df = pd.read_csv(file_path, sep='\t', encoding='cp949')

    # 3. 필요한 컬럼의 데이터 타입 확인 및 변환
    # '승차총승객수'와 '하차총승객수'가 숫자로 되어 있는지 확인하고, 아니면 숫자로 변환합니다.
    # errors='coerce' 옵션을 사용하여 숫자로 변환할 수 없는 값은 NaN으로 만듭니다.
    df['승차총승객수'] = pd.to_numeric(df['승차총승객수'], errors='coerce')
    df['하차총승객수'] = pd.to_numeric(df['하차총승객수'], errors='coerce')

    # NaN 값 (변환 실패한 값)이 있다면 0으로 채웁니다.
    df = df.fillna(0)

    # 4. 노선별 총 승객수 계산
    # '승차총승객수'와 '하차총승객수'를 더하여 '총승객수' 컬럼을 만듭니다.
    df['총승객수'] = df['승차총승객수'] + df['하차총승객수']


    # Group the DataFrame by '역명' (Station Name) and sum the '총승객수' for each station.
    total_passengers_by_station = df.groupby('역명')['총승객수'].sum()

    # Get the top 5 stations with the highest total passenger count.
    top_5_stations = total_passengers_by_station.nlargest(5)

    # Print the result.
    print("가장 붐비는 역 TOP 5 (승차 및 하차 총합 기준):")
    print(top_5_stations)

    # --- 막대 그래프 생성 ---
    # 폰트 설정 (한글 깨짐 방지)
    # 시스템에 'Malgun Gothic' 폰트가 없으면 다른 한글 폰트 (예: 'AppleGothic', 'NanumGothic')로 변경하거나
    # 폰트 파일을 다운로드하여 설정해야 합니다.
    plt.rcParams['font.family'] = 'NanumGothic' # For Windows
    plt.rcParams['axes.unicode_minus'] = False # Prevents minus sign from breaking font

    # Create the plot figure and axes
    plt.figure(figsize=(12, 7)) # Set the figure size for better readability
    top_5_stations.plot(kind='bar', color='lightcoral') # Plot as a bar chart with a different color

    # Set the title and axis labels
    plt.title('가장 붐비는 지하철 역 TOP 5 (2025년 5월)', fontsize=18, pad=20)
    plt.xlabel('역명', fontsize=14)
    plt.ylabel('총 승객수', fontsize=14)

    # Format Y-axis labels to display with commas for thousands
    plt.ticklabel_format(axis='y', style='plain') # Use plain style instead of scientific notation
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

    # Add passenger counts on top of the bars
    for index, value in enumerate(top_5_stations):
        plt.text(index, value, f'{value:,.0f}', ha='center', va='bottom', fontsize=11)

    # Rotate X-axis labels for better readability if station names are long
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(fontsize=12)

    plt.grid(axis='y', linestyle='--', alpha=0.7) # Add a grid for easier reading
    plt.tight_layout() # Adjust layout to prevent labels from overlapping
    plt.show() # Display the plot

except FileNotFoundError:
    print("오류: 'CARD_SUBWAY_MONTH_202505.txt' 파일을 찾을 수 없습니다. 파일이 올바른 위치에 있는지 확인해주세요.")
except Exception as e:
    print(f"데이터 처리 중 오류가 발생했습니다: {e}")